In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
conf = SparkConf().setMaster('local').setAppName('241204_01_RDD_API')
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/04 11:32:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/04 11:32:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local appName=241204_01_RDD_API>

In [7]:
foods = spark.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [8]:
# 음식별 개수 세기 
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [9]:
# 상위 3개만 가져와
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [10]:
# 처음 1개 가져오기
foods.first()

'짜장면'

In [11]:
# RDD 개수
foods.count()

12

In [13]:
# 중복을 제거한 목록
d_f = foods.distinct()
d_f.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [14]:
# 워커노드에서 실행하는 기능
foods.foreach(lambda x: print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


## Narrow Operation 1:1 연산
filter(), map(), flatMap(), sample(), union()

In [15]:
sample_rdd = spark.parallelize([1,2,3,4,5])

In [19]:
sample_rdd2 = sample_rdd.map(lambda x: x+2) # d+2, 2+2, 3+2....
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [20]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [21]:
movie_RDD = spark.parallelize(movies)

In [23]:
mapMovies = movie_RDD.map(lambda x: x.split(' '))
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [24]:
f_mapMovies = movie_RDD.flatMap(lambda x: x.split(' '))
f_mapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [25]:
filteredMovies = f_mapMovies.filter(lambda x : x!='매트릭스')
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [26]:
# 집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [27]:
# intersection -> 교집합
n1_n2 = num1.intersection(num2)
n1_n2.collect()

[4, 5]

In [28]:
# union, subtract
n1_n2_u = num1.union(num2)
n1_n2_u.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [29]:
n1_n2_s = num1.subtract(num2)
n1_n2_s.collect()

[2, 1, 3]

# sample(withReplacement, fraction, seed)<br>
## 일부를 샘플링해서 추출.<br>
withReplacement : 비복원, 복원 추출<br>
fraction : 기대값<br>
seed : 난수추출을 위한 시드값 - 재현 가능<br>

In [30]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [31]:
numlist.sample(True, 0.3).collect()

[1, 4, 7, 9]

In [32]:
numlist.sample(True, 0.5).collect()

[2, 5, 5, 8, 9, 10, 10]

In [33]:
numlist.sample(False, 0.7).collect() # 재현이 안되는 샘플

[1, 2, 4, 5, 6, 7, 9]

In [34]:
numlist.sample(True, 0.5, seed=42).collect() # 재현이 가능함

[5, 6, 6]

# wide transformation
groupby reduce

In [35]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [36]:
foodsGroup = foods.groupBy(lambda x : x[0])
res = foodsGroup.collect()

In [37]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [38]:
spark.stop()